In [3]:
import gymnasium as gym
import math
import random
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple, deque
from itertools import count

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

env = gym.make("CartPole-v1")

# matplotlib 설정
is_ipython = 'inline' in matplotlib.get_backend()
if is_ipython:
    from IPython import display

plt.ion()

# GPU를 사용할 경우
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [4]:
torch.cuda.is_available()

True

In [8]:
!echo $CONDA_PREFIX

/home/mlcf/anaconda3/envs/rl


In [5]:
print(torch.utils.cpp_extension.CUDA_HOME)

AttributeError: module 'torch.utils' has no attribute 'cpp_extension'